In [ ]:
# Connect to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instal library dependencies
%pip install -q ultralytics supervision mlflow pyngrok

In [ ]:
# === Extract Dataset ===
# Tentukan path ke file .zip Anda di Google Drive
ZIP_FILE_PATH = '/content/drive/MyDrive/snapcal/datasets/snapcal-1.zip'

# Tentukan di mana kita akan mengekstrak dataset di lingkungan Colab (untuk kecepatan akses)
EXTRACT_DESTINATION = '/content/dataset'

print(f"🔄 Mengekstrak {ZIP_FILE_PATH}...")
# Gunakan perintah 'unzip' untuk mengekstrak file. Opsi '-q' membuatnya senyap.
!unzip -q "{ZIP_FILE_PATH}" -d "{EXTRACT_DESTINATION}"
print(f"✅ Dataset berhasil diekstrak ke {EXTRACT_DESTINATION}")

In [ ]:
import os
import mlflow
from ultralytics import YOLO, settings

# --- PERUBAHAN PENTING ---
# Path ke file data.yaml sekarang menunjuk ke folder yang telah diekstrak di Colab
DATASET_YAML_PATH = '/content/dataset/data.yaml'

# --- Pengaturan MLflow ---
settings.update({'mlflow': False})
mlflow.set_tracking_uri('file:./mlruns')
mlflow.set_experiment("SnapCal Component Detection - Colab")

# --- Mulai Sesi Pelatihan ---
with mlflow.start_run() as run:
    run_id = run.info.run_id
    print(f"🚀 Memulai pelatihan Colab - Run ID: {run_id}")

    # --- Parameter Pelatihan ---
    params = {
        "epochs": 50,
        "batch": 16,
        "imgsz": 640,
        "device": 0,
        "amp": True,
        "workers": 2,
        "patience": 10
    }
    mlflow.log_params(params)
    print(f"🎯 Parameter Pelatihan: {params}")

    # --- Muat Model & Latih ---
    model = YOLO("yolov8n.pt")

    results = model.train(
        data=DATASET_YAML_PATH,
        **params,
        # Simpan hasil pelatihan ke Google Drive agar tidak hilang
        project='/content/drive/MyDrive/snapcal/training_runs',
        name=f'run_{run_id}'
    )

    # --- Catat Metrik & Model ---
    final_metrics = {
        "mAP50-95": results.box.map,
        "mAP50": results.box.map50,
        "precision": results.box.mp,
        "recall": results.box.mr
    }
    mlflow.log_metrics(final_metrics)
    print(f"📊 Metrik Final: {final_metrics}")

# --- Blok Kode yang Telah Diperbaiki ---
print("\n--- 💾 Menyimpan model terbaik sebagai artifact ---")

# Objek 'results' berisi path ke direktori tempat hasil pelatihan disimpan
training_run_path = results.save_dir

# Path lengkap ke file bobot model terbaik
best_model_path = os.path.join(training_run_path, 'weights/best.pt')

# Periksa apakah file tersebut ada sebelum mencatatnya
if os.path.exists(best_model_path):
    # Gunakan mlflow.log_artifact untuk menyimpan file apa pun.
    # Argumen kedua, "model", akan menempatkannya di dalam subfolder 'model' di MLflow UI.
    mlflow.log_artifact(best_model_path, "model")
    print(f"✅ Model terbaik dari '{best_model_path}' berhasil dicatat sebagai artifact.")
else:
    print(f"⚠️ Peringatan: File model terbaik tidak ditemukan di '{best_model_path}'.")